# utils

In [1]:
import vectorbt as vbt

from vectorbt.utils import checks, config, decorators

In [2]:
import numpy as np
import pandas as pd
from numba import njit

In [3]:
v1 = 0
a1 = np.array([1])
a2 = np.array([1, 2, 3])
a3 = np.array([[1, 2, 3]])
a4 = np.array([[1], [2], [3]])
a5 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sr1 = pd.Series([1], index=pd.Index(['x1'], name='i1'))
print(sr1)
sr2 = pd.Series([1, 2, 3], index=pd.Index(['x2', 'y2', 'z2'], name='i2'))
print(sr2)
df1 = pd.DataFrame(
    [[1]], 
    index=pd.Index(['x3'], name='i3'), 
    columns=pd.Index(['a3'], name='c3'))
print(df1)
df2 = pd.DataFrame(
    [[1], [2], [3]], 
    index=pd.Index(['x4', 'y4', 'z4'], name='i4'), 
    columns=pd.Index(['a4'], name='c4'))
print(df2)
df3 = pd.DataFrame(
    [[1, 2, 3]], 
    index=pd.Index(['x5'], name='i5'), 
    columns=pd.Index(['a5', 'b5', 'c5'], name='c5'))
print(df3)
df4 = pd.DataFrame(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 
    index=pd.Index(['x6', 'y6', 'z6'], name='i6'), 
    columns=pd.Index(['a6', 'b6', 'c6'], name='c6'))
print(df4)

multi_i = pd.MultiIndex.from_arrays([['x7', 'y7', 'z7'], ['x8', 'y8', 'z8']], names=['i7', 'i8']) 
multi_c = pd.MultiIndex.from_arrays([['a7', 'b7', 'c7'], ['a8', 'b8', 'c8']], names=['c7', 'c8'])
df5 = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=multi_i, columns=multi_c)
print(df5)

i1
x1    1
dtype: int64
i2
x2    1
y2    2
z2    3
dtype: int64
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  4  5  6
z7 z8  7  8  9


## config

In [4]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen=False)
conf['b']['d'] = 2

conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen=True)
conf['a'] = 2

try:
    conf['d'] = 2
except Exception as e:
    print(e)

try:
    conf.update(d=2)
except Exception as e:
    print(e)
    
conf.update(d=2, force_update=True)

"Key 'd' is not valid"
"Key 'd' is not valid"


In [5]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, read_only=True)

try:
    conf['a'] = 2
except Exception as e:
    print(e)
    
try:
    del conf['a']
except Exception as e:
    print(e)
    
try:
    conf.pop('a')
except Exception as e:
    print(e)
    
try:
    conf.popitem()
except Exception as e:
    print(e)
    
try:
    conf.clear()
except Exception as e:
    print(e)
    
try:
    conf.update(a=2)
except Exception as e:
    print(e)
    
print(conf.merge_with(dict(b=dict(d=2))))

Config is read-only
Config is read-only
Config is read-only
Config is read-only
Config is read-only
Config is read-only
{'b': {'c': 1, 'd': 2}, 'a': 0}


In [6]:
print(config.merge_dicts({'a': 1}, {'b': 2}))
print(config.merge_dicts({'a': 1}, {'a': 2}))
print(config.merge_dicts({'a': {'b': 2}}, {'a': {'c': 3}}))
print(config.merge_dicts({'a': {'b': 2}}, {'a': {'b': 3}}))

{'a': 1, 'b': 2}
{'a': 2}
{'a': {'b': 2, 'c': 3}}
{'a': {'b': 3}}


In [7]:
class H(config.Configured):
    def __init__(self, a, b=2, **kwargs):
        super().__init__(a=a, b=b, **kwargs)
        
print(H(1).config)
print(H(1).copy(b=3).config)
print(H(1).copy(c=4).config)

{'a': 1, 'b': 2}
{'a': 1, 'b': 3}
{'a': 1, 'b': 2, 'c': 4}


## decorators

In [8]:
class G():
    @decorators.class_or_instancemethod
    def g(self_or_cls):
        if isinstance(self_or_cls, type):
            print("class")
        else:
            print("instance")
            
G.g()
G().g()

class
instance


In [9]:
class G():
    @decorators.cached_property(hello="world", hello2="world2")
    def cache_me(self): return np.random.uniform(size=(10000, 10000))
    
G.cache_me.kwargs

{'hello': 'world', 'hello2': 'world2'}

In [10]:
g = G()

In [11]:
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.3 s, sys: 276 ms, total: 1.58 s
Wall time: 1.69 s
CPU times: user 33 µs, sys: 1e+03 ns, total: 34 µs
Wall time: 37.9 µs


In [12]:
dir(g)

['__cached_cache_me',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [13]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.37 s, sys: 439 ms, total: 1.81 s
Wall time: 2.6 s
CPU times: user 67 µs, sys: 231 µs, total: 298 µs
Wall time: 304 µs


In [14]:
G.cache_me.clear_cache(g)
G.cache_me.disabled = True
%time _ = g.cache_me
%time _ = g.cache_me
G.cache_me.disabled = False

CPU times: user 1.23 s, sys: 350 ms, total: 1.58 s
Wall time: 1.64 s
CPU times: user 424 µs, sys: 565 µs, total: 989 µs
Wall time: 6.65 ms


In [15]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append((g, 'cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.28 s, sys: 311 ms, total: 1.59 s
Wall time: 1.74 s
CPU times: user 1.28 s, sys: 325 ms, total: 1.6 s
Wall time: 1.73 s


In [16]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('cache_me')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.44 s, sys: 379 ms, total: 1.82 s
Wall time: 2.38 s
CPU times: user 1.3 s, sys: 340 ms, total: 1.64 s
Wall time: 1.78 s


In [17]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(g)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.24 s, sys: 311 ms, total: 1.55 s
Wall time: 1.59 s
CPU times: user 1.22 s, sys: 302 ms, total: 1.52 s
Wall time: 1.56 s


In [18]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(G)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 296 ms, total: 1.52 s
Wall time: 1.56 s
CPU times: user 1.23 s, sys: 304 ms, total: 1.54 s
Wall time: 1.58 s


In [19]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('G')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.24 s, sys: 312 ms, total: 1.55 s
Wall time: 1.59 s
CPU times: user 1.22 s, sys: 299 ms, total: 1.52 s
Wall time: 1.55 s


In [20]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('g')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.24 s, sys: 308 ms, total: 1.55 s
Wall time: 1.61 s
CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 30 µs


In [21]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 297 ms, total: 1.53 s
Wall time: 1.56 s
CPU times: user 1.21 s, sys: 290 ms, total: 1.5 s
Wall time: 1.51 s


In [22]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.22 s, sys: 295 ms, total: 1.51 s
Wall time: 1.54 s
CPU times: user 1.21 s, sys: 298 ms, total: 1.51 s
Wall time: 1.53 s


In [23]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2', 'hello3': 'world3'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.22 s, sys: 294 ms, total: 1.51 s
Wall time: 1.54 s
CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 31.9 µs


In [24]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.22 s, sys: 294 ms, total: 1.52 s
Wall time: 1.54 s
CPU times: user 1.23 s, sys: 304 ms, total: 1.53 s
Wall time: 1.57 s


In [25]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append((g, 'cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 281 ms, total: 1.49 s
Wall time: 1.5 s
CPU times: user 37 µs, sys: 0 ns, total: 37 µs
Wall time: 38.9 µs


In [26]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('cache_me')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 288 ms, total: 1.49 s
Wall time: 1.52 s
CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 29.3 µs


In [27]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(g)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 286 ms, total: 1.5 s
Wall time: 1.51 s
CPU times: user 26 µs, sys: 1 µs, total: 27 µs
Wall time: 29.1 µs


In [28]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(G)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 288 ms, total: 1.49 s
Wall time: 1.5 s
CPU times: user 27 µs, sys: 1e+03 ns, total: 28 µs
Wall time: 30 µs


In [29]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('G')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 292 ms, total: 1.5 s
Wall time: 1.52 s
CPU times: user 27 µs, sys: 1e+03 ns, total: 28 µs
Wall time: 30 µs


In [30]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append({'hello': 'world'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 296 ms, total: 1.51 s
Wall time: 1.53 s
CPU times: user 28 µs, sys: 1e+03 ns, total: 29 µs
Wall time: 31 µs


In [31]:
class G():
    @decorators.cached_method(hello="world", hello2="world2")
    def cache_me(self, a): return np.random.uniform(size=(10000, 10000)) * a

G.cache_me.kwargs

{'hello': 'world', 'hello2': 'world2'}

In [32]:
g = G()

In [33]:
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)

CPU times: user 1.31 s, sys: 295 ms, total: 1.61 s
Wall time: 1.64 s
CPU times: user 34 µs, sys: 1e+03 ns, total: 35 µs
Wall time: 37.2 µs


In [34]:
dir(g)

['__cached_cache_me',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [35]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me(2)

CPU times: user 1.31 s, sys: 294 ms, total: 1.61 s
Wall time: 1.63 s


In [36]:
G.cache_me.clear_cache(g)
G.cache_me.disabled = True
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
G.cache_me.disabled = False

CPU times: user 1.3 s, sys: 289 ms, total: 1.59 s
Wall time: 1.61 s
CPU times: user 34 µs, sys: 1 µs, total: 35 µs
Wall time: 36 µs


In [37]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(g.cache_me)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 285 ms, total: 1.6 s
Wall time: 1.62 s
CPU times: user 1.31 s, sys: 281 ms, total: 1.59 s
Wall time: 1.6 s


In [38]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append((g, 'cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.32 s, sys: 292 ms, total: 1.61 s
Wall time: 1.62 s
CPU times: user 1.3 s, sys: 281 ms, total: 1.58 s
Wall time: 1.59 s


In [39]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('cache_me')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 294 ms, total: 1.61 s
Wall time: 1.62 s
CPU times: user 1.36 s, sys: 297 ms, total: 1.66 s
Wall time: 1.7 s


In [40]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(g)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.35 s, sys: 303 ms, total: 1.65 s
Wall time: 1.69 s
CPU times: user 1.3 s, sys: 286 ms, total: 1.59 s
Wall time: 1.6 s


In [41]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(G)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.36 s, sys: 300 ms, total: 1.66 s
Wall time: 1.69 s
CPU times: user 1.3 s, sys: 281 ms, total: 1.58 s
Wall time: 1.59 s


In [42]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('G')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 285 ms, total: 1.59 s
Wall time: 1.61 s
CPU times: user 1.31 s, sys: 284 ms, total: 1.59 s
Wall time: 1.61 s


In [43]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('g')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.32 s, sys: 302 ms, total: 1.62 s
Wall time: 1.65 s
CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37.9 µs


In [44]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.32 s, sys: 287 ms, total: 1.6 s
Wall time: 1.62 s
CPU times: user 1.32 s, sys: 289 ms, total: 1.6 s
Wall time: 1.62 s


In [45]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.3 s, sys: 288 ms, total: 1.59 s
Wall time: 1.6 s
CPU times: user 1.3 s, sys: 278 ms, total: 1.58 s
Wall time: 1.59 s


In [46]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2', 'hello3': 'world3'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 287 ms, total: 1.59 s
Wall time: 1.62 s
CPU times: user 36 µs, sys: 1 µs, total: 37 µs
Wall time: 39.1 µs


In [47]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 294 ms, total: 1.6 s
Wall time: 1.61 s
CPU times: user 1.32 s, sys: 291 ms, total: 1.61 s
Wall time: 1.62 s


In [48]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append((g, 'cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 289 ms, total: 1.6 s
Wall time: 1.62 s
CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37 µs


In [49]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('cache_me')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 288 ms, total: 1.6 s
Wall time: 1.61 s
CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37.9 µs


In [50]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(g)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 283 ms, total: 1.6 s
Wall time: 1.62 s
CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 37 µs


In [51]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(G)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 289 ms, total: 1.6 s
Wall time: 1.62 s
CPU times: user 35 µs, sys: 1e+03 ns, total: 36 µs
Wall time: 37.2 µs


In [52]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('G')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 282 ms, total: 1.59 s
Wall time: 1.63 s
CPU times: user 36 µs, sys: 1 µs, total: 37 µs
Wall time: 37.9 µs


In [53]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append({'hello': 'world'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 299 ms, total: 1.61 s
Wall time: 1.64 s
CPU times: user 37 µs, sys: 0 ns, total: 37 µs
Wall time: 40.1 µs


In [54]:
# Non-hashable arguments won't cache
%time _ = g.cache_me(np.asarray(2))
%time _ = g.cache_me(np.asarray(2))

CPU times: user 1.32 s, sys: 228 ms, total: 1.55 s
Wall time: 1.57 s
CPU times: user 1.7 s, sys: 869 ms, total: 2.56 s
Wall time: 3.55 s


In [55]:
class A:
    @decorators.custom_property(some_key=0)
    def a(self): pass

class B:
    @decorators.cached_property(some_key=0, child_cls=A)
    def a(self): pass
    
    @decorators.custom_method(some_key=1)
    def b(self): pass
    
class C:
    @decorators.cached_method(some_key=0, child_cls=B)
    def b(self): pass
    
    @decorators.custom_property(some_key=1)
    def c(self): pass

In [56]:
str(decorators.traverse_attr_kwargs(C))

"{'b': {'some_key': 0, 'child_cls': <class '__main__.B'>, 'child_attrs': {'a': {'some_key': 0, 'child_cls': <class '__main__.A'>, 'child_attrs': {'a': {'some_key': 0}}}, 'b': {'some_key': 1}}}, 'c': {'some_key': 1}}"

In [57]:
decorators.traverse_attr_kwargs(C, key='some_key')

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {'a': {'some_key': 0}}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

In [58]:
decorators.traverse_attr_kwargs(C, key='some_key', value=1)

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

In [59]:
decorators.traverse_attr_kwargs(C, key='some_key', value=(0, 1))

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {'a': {'some_key': 0}}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

## checks

In [60]:
print(checks.is_series(v1))
print(checks.is_series(a1))
print(checks.is_series(sr1))
print(checks.is_series(df1))

False
False
True
False


In [61]:
print(checks.is_frame(v1))
print(checks.is_frame(a1))
print(checks.is_frame(sr1))
print(checks.is_frame(df1))

False
False
False
True


In [62]:
print(checks.is_pandas(v1))
print(checks.is_pandas(a1))
print(checks.is_pandas(sr1))
print(checks.is_pandas(df1))

False
False
True
True


In [63]:
print(checks.is_array(v1))
print(checks.is_array(a1))
print(checks.is_array(sr1))
print(checks.is_array(df1))

False
True
True
True


In [64]:
print(checks.is_numba_func(lambda x: x))
print(checks.is_numba_func(njit(lambda x: x)))

False
True


In [65]:
print(checks.is_hashable(2))
print(checks.is_hashable(np.asarray(2)))

True
False


In [66]:
checks.assert_in(0, (0, 1))

In [67]:
checks.assert_numba_func(njit(lambda x: x))

In [68]:
checks.assert_not_none(v1)

In [69]:
checks.assert_type(v1, int)
checks.assert_type(a1, np.ndarray)
checks.assert_type(sr1, (np.ndarray, pd.Series))

In [70]:
checks.assert_type_equal(v1, v1)
checks.assert_type_equal(a1, a2)
checks.assert_type_equal(sr1, sr1)
checks.assert_type_equal(df1, df2)

In [71]:
checks.assert_dtype(a1, np.int)

In [72]:
checks.assert_dtype_equal(v1, a1)
checks.assert_dtype_equal(a1, df1)
checks.assert_dtype_equal(df1, df2)
checks.assert_dtype_equal(df2, df3)

In [73]:
checks.assert_ndim(v1, 0)
checks.assert_ndim(a1, 1)
checks.assert_ndim(df1, 2)

In [74]:
checks.assert_len_equal([[1]], [[2]])

In [75]:
checks.assert_shape_equal(a1, sr1)
checks.assert_shape_equal(df2, df4, axis=0)
checks.assert_shape_equal(df3, df4, axis=1)
checks.assert_shape_equal(df2, df3, axis=(0, 1))

In [76]:
checks.assert_index_equal(df3.index, df3.index)

In [77]:
checks.assert_meta_equal(df3, df3)

In [78]:
checks.assert_array_equal(df3, df3)

In [79]:
checks.assert_level_not_exists(df3.columns, 'a')